In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import *

In [0]:
# Set parquet configuration to interpret binary byte array as string
spark = SparkSession.builder.config('spark.sql.parquet.binaryAsString', 'true') \
            .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
            .appName("OMS") \
            .getOrCreate()

# sc = spark.sparkContext
# sc.setLogLevel("ERROR")

In [0]:
spark

Out[3]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 spark://10.215.253.154:7077 
 AppName 
 Databricks Shell

**Long Song Popularity Data**

In [0]:
database = 'oms'
collection = 'song_popularity_predictions_data'
user_name = 'user'
password = 'user'
address = 'oms-cluster.0navm.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [0]:
df_song_pop = spark.read.format("mongo").option("uri",connection_string).load()

In [0]:
df_song_pop.columns

Out[36]: ['_id', 'artist_id', 'artist_name', 'prediction', 'song_id', 'title']

**Loading Play count Data Set**

In [0]:
database = 'oms'
collection = 'play_counts'
user_name = 'user'
password = 'user'
address = 'oms-cluster.0navm.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [0]:
df_playcounts_ = spark.read.format("mongo").option("uri",connection_string).load()
df_playcounts =df_playcounts_.select('play_count', 'song_id', 'user_id').cache()

In [0]:
df_playcounts.display(1)

Load song Genre

In [0]:
database = 'oms'
collection = 'song_genre'
user_name = 'user'
password = 'user'
address = 'oms-cluster.0navm.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [0]:
df_genre_ = spark.read.format("mongo").option("uri",connection_string).load()
df_genre = df_genre_.select('genre','song_id').cache()

In [0]:
df_genre.display(10)

# Joining the genre to playcount data

In [0]:
df_genre_playcounts = df_genre.join(df_playcounts,'song_id' ,'inner').cache()

In [0]:
df_genre_playcounts.display(10)

song_id,genre,play_count,user_id
SOAAZPG12A6D4F8D8B,Pop Rock,1,2199769200b689c44cd8442f51e4f82047ecf679
SOAAZPG12A6D4F8D8B,Pop Rock,1,4af4db5cdd51f8cdd0917cc51fa9f53e7bcb031b
SOAAZPG12A6D4F8D8B,Pop Rock,1,3b4ad4a00c1a8edb790435e9ea4ed6f354445f94
SOAAZPG12A6D4F8D8B,Pop Rock,1,3a73046fe1a5aae7d8fa6173cbd053db85c00988
SOAAZPG12A6D4F8D8B,Pop Rock,1,6fd7851e1d02a3aeaf90855bba34e72f9d256173
SOAAZPG12A6D4F8D8B,Pop Rock,1,4b49956d0855007f80462b498f41e2a5b93a6f6e
SOAAZPG12A6D4F8D8B,Pop Rock,1,f3fd89959b9a004eb8bc31c1e41f2b756a39927a
SOAAZPG12A6D4F8D8B,Pop Rock,1,c6535de39bc7a062daf12a71be78e37e090dd3c6
SOAAZPG12A6D4F8D8B,Pop Rock,1,bb84b605789d898993e2c6fbda4d57a8bc8da369
SOAAZPG12A6D4F8D8B,Pop Rock,1,24145dac630d4f7c33544a12c054affd85358537


In [0]:
df_genre_playcounts_grouped =spark.sql("select genre, user_id, AVG(play_count) as avg_play_count from df_genre_playcounts group by genre, user_id")

In [0]:
df_genre_playcounts_grouped.write.saveAsTable('default.df_genre_playcounts_grouped')

average playcount per genre

In [0]:
top_genre = spark.sql("select genre, user_id, avg_play_count, rank() over (partition by genre order by avg_play_count desc ) as rank_ from df_genre_playcounts_grouped")

In [0]:
top_genre.display(10)

song_id,genre,user_id,play_count,rank_
SOYFYHE12A8C142082,International,dcf6a11b2fea3af24fb001f504851ed23aae9965,686,1
SOZGGXU12A67ADD3E4,International,762c6fa3791c68f5b530b91a3413a8f480df2099,407,2
SOAJDBZ12A6D4FA3C8,International,e3d47f8d33e91da3c476577cdf0b5dc3fe1e2d2a,296,3
SOZGGXU12A67ADD3E4,International,8cbcb30da89e66b46136dca4f3ab5d70893f7db8,229,4
SOAJDBZ12A6D4FA3C8,International,e3f6ac98a2b45f6ac0b0b1e118774828d8be2029,227,5
SOMECGO12AB0186D31,International,d9b255c131ba531ef58a393e3c5fb3f43f2dc53c,224,6
SOCSAOO12AB017E850,International,3cc0475a6842e690fc9ed3dcf232ead0fd01e0c5,167,7
SOUWNIQ12A67ADE5B6,International,64074f44ffae272464a75b3ca3fbcddcec95eda6,160,8
SOSBZQZ12A6D4FA577,International,52eeb1ecde85689adc8bd3c6f393a6ff04f9aaba,156,9
SOAJDBZ12A6D4FA3C8,International,6aa76847a05f6f10fcd7c1bf4ed6f6ab1ac4e90e,150,10


In [0]:
spark.sql('drop table default.top_genre_sql')
top_genre.write.saveAsTable('default.top_genre_sql')

In [0]:
top_genre.columns

Out[21]: ['genre', 'user_id', 'avg_play_count', 'rank_']

Select the rank1

In [0]:
top_genre_final= spark.sql('select genre, user_id, avg_play_count\
                        from top_genre_sql where rank_=1')

In [0]:
top_genre_final.display(10)

genre,user_id,avg_play_count
Pop Rock,cdc1e4e896757761cfa542e31fc7ea0085a4d519,743.0
Electronic,780913e6a5d61405f4653aef0231876e6721120b,1369.0
Rap,296e5769223ae111e530d96c8bcf95fd63d8af9b,475.0
Reggae,8665ae2a21ff60a45bf638c941a61a0cb75d7687,217.0
Avant Garde,c53ac12f0b9396ab3f44f7b521d5925b10e991d3,13.0
Folk,9ae2abce15efa19998984882900850b927ebf7e1,496.0
Holiday,d4ca0ff508531f8e2f53407924f12a3adfc72a86,64.0
RnB,98cb9a3feb48d15a8328ce16f1b32609698e11f2,835.0
Stage,78d77cba6776c72ca78a52aa66f2526eabd86ea7,141.0
International,762c6fa3791c68f5b530b91a3413a8f480df2099,407.0


In [0]:
df_first_page = df_playcounts.select('song_id', 'user_id', 'play_count').join(top_genre_final, 'user_id', 'inner').dropDuplicates()

In [0]:
df_first_page.display()

user_id,song_id,play_count,genre,avg_play_count
780913e6a5d61405f4653aef0231876e6721120b,SOBQJBF12AF72A2EE6,2,Electronic,1369.0
780913e6a5d61405f4653aef0231876e6721120b,SOHXGXU12A8C1413EB,2,Electronic,1369.0
780913e6a5d61405f4653aef0231876e6721120b,SOIGZMC12A6D4F979B,1,Electronic,1369.0
780913e6a5d61405f4653aef0231876e6721120b,SOKNMJE12A67AE0421,1369,Electronic,1369.0
780913e6a5d61405f4653aef0231876e6721120b,SOKWNPY12A8C13A8FA,1,Electronic,1369.0
780913e6a5d61405f4653aef0231876e6721120b,SOKWVQL12A67ADF7DA,20,Electronic,1369.0
780913e6a5d61405f4653aef0231876e6721120b,SOOQPPL12AB017D2E3,29,Electronic,1369.0
780913e6a5d61405f4653aef0231876e6721120b,SOOWJZM12A6D4F7995,1,Electronic,1369.0
780913e6a5d61405f4653aef0231876e6721120b,SOSFOFO12AF72AD198,2,Electronic,1369.0
780913e6a5d61405f4653aef0231876e6721120b,SOTVFIU12AC46878B7,1,Electronic,1369.0


Adding the song name to the Data

In [0]:
df_first_page_final = df_song_pop.select('song_id', 'title', 'prediction').join(df_first_page, 'song_id', 'inner')

In [0]:
df_first_page_final.display()

song_id,title,prediction,user_id,play_count,genre,avg_play_count
SOAGIAY12AB0183D8F,Run Me Down,0.7473401652954215,5a44bfcaa1081e8b36ff76ec32c2ef73425ad2e9,5,Vocal,210.0
SOAGIAY12AB0183D8F,Run Me Down,0.7473401652954215,78d77cba6776c72ca78a52aa66f2526eabd86ea7,9,Stage,141.0
SOUJVIT12A8C1451C1,Savior,0.9181823430861558,5a44bfcaa1081e8b36ff76ec32c2ef73425ad2e9,2,Vocal,210.0
SOUJVIT12A8C1451C1,Savior,0.9181823430861558,d8e6fa08d73821f305b9a3af1cf1e0a704473d82,1,Country,1179.3333333333333
SOFRBUK12A67AD80D7,Precious,0.7493403173877817,ca9fb6d60f3261f783936a7ff48df2df176d5b3a,1,Comedy Spoken,219.0
SOTLLRE12A6D4F8EFF,Breaking Down The Walls Of Heartache,0.3943948561005826,6c966bdfc8ef4699fe1814e18573419b725d94e6,71,Easy Listening,97.0
SOIYNKK12A6D4F7A54,Esse Coracao (Este Corazón),0.7966929516579855,f0eb3676a2b775f67f3e6ddc3ad43006f50d4aca,1,New Age,675.0
SONUNII12AC4689BD5,Nuevo,0.42512465831305757,cdc1e4e896757761cfa542e31fc7ea0085a4d519,2,Pop Rock,743.0
SOPXKYD12A6D4FA876,Yellow,0.9203585279112777,d8e6fa08d73821f305b9a3af1cf1e0a704473d82,1,Country,1179.3333333333333
SOTLURY12AB0183C93,Mr Wendal (Edit),0.0,42b31eaf0cbaa10e593438e2858ce74cdfa01191,4,Jazz,242.0


In [0]:
#spark.sql('drop table default.df_first_page_data')
df_first_page_final.write.saveAsTable('default.df_first_page_final')

# For page-2

In [0]:
database = 'oms'
collection = 'recommendations'
user_name = 'user'
password = 'user'
address = 'oms-cluster.0navm.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [0]:
df_song_emb = spark.read.format("mongo").option("uri",connection_string).load()

In [0]:
df_song_emb.select('user_id-num',explode(df_song_emb.recommendations)).display()

user_id-num,col
22,"List(326979, 21.067590713500977)"
22,"List(294093, 19.417848587036133)"
22,"List(158233, 14.881278038024902)"
22,"List(290778, 14.291248321533203)"
22,"List(1064, 12.689688682556152)"
22,"List(368156, 11.96796703338623)"
22,"List(251152, 11.468047142028809)"
22,"List(127025, 11.142863273620605)"
22,"List(367711, 10.853818893432617)"
22,"List(316469, 10.562719345092773)"
